In Python functions are first-class entities which allows us to construct functions taking another functions as their arguments. Such functions operating on the other functions are usually called higher-order functions. It's a quite powerful tool for constructing more general solutions of some existing problems.

### Example

In [1]:
def makeLog(file):
    
    def log(entry):
        with open(file, 'a') as f:
            f.write(entry + '\n')
    
    return log

In [2]:
testLog = makeLog('log.txt')

In [3]:
testLog('Don\'t')
testLog('Panic')

In [4]:
entries = open('log.txt', 'r').read()
print(entries)

Don't
Panic
Don't
Panic



### Decorators

We want to compare the time it takes to compute the 30th Fibonacci number by using the naive recursive approach and making it in an iterative way. 

In [5]:
def fib(n):
    a, b = 1, 1
    for i in range(n):
        a, b = b, a + b
    return a

def ffib(n):
    return 1 if n in (0, 1) else ffib(n-1) + ffib(n-2)

In [6]:
from time import time

t = time()
ffib(30)
print(f'execution time: {time() - t}')

t = time()
fib(30)
print(f'execution time: {time() - t}')

execution time: 0.24158358573913574
execution time: 4.8160552978515625e-05


If some code is used more than once, it's a good evidence there's something to generalize there.

In [7]:
def timed(func):
    
    def inner(*args, **kwargs):
        from time import time
        t = time()
        res = func(*args, **kwargs)
        print(f'execution time: {time() - t}')
        return res
    
    return inner

In [8]:
tfib = timed(fib)
tffib = timed(ffib)

In [9]:
tfib(30)
tffib(30)

execution time: 3.814697265625e-06
execution time: 0.24297046661376953


1346269

`*args` means that all the arguments given to the function after those explicitly specified in the brackets should be packed into a list named `args`. `**kwargs` means that all the keyword arguments given to the function which are not among those written explicitly in the definition should be packed into a dictionary called `kwargs`. When used outside of a function definition, these stars are used to unpack these structures.

The introduce wrapper that takes a function and returns the same function that prints out its name before being executed.

In [10]:
def introduce(func):
    
    def inner(*args, **kwargs):
        print(func.__name__)
        return func(*args, **kwargs)
    
    return inner

In [11]:
@introduce
def id(x):
    '''Identity function'''
    return x

In [12]:
print(id(20))

id
20


Adding some enhancing feature to an existing function is exactly what decorators are used for.

In [13]:
def cached(func):
    func.cache = {}
    
    def inner(*args, **kwargs):
        key = args, tuple(kwargs.items())
        
        if not key in func.cache:
            func.cache[key] = func(*args, **kwargs)
            
        return func.cache[key]
    
    return inner

In [14]:
@cached
def cffib(n):
    return 1 if n in (0, 1) else cffib(n-1) + cffib(n-2)

tcffib = timed(cffib)

In [15]:
tfib(300)
tffib(30)
tcffib(300)

execution time: 1.811981201171875e-05
execution time: 0.24740147590637207
execution time: 0.0005671977996826172


359579325206583560961765665172189099052367214309267232255589801

We've created the cache dictionary as a property of the `func` function.

### Subtleties of decorating

In [16]:
print(f'{help(id)}\n----\n{id.__name__}')

Help on function inner in module __main__:

inner(*args, **kwargs)

None
----
inner


Certainly it's not what we expected to see. Fix:

In [17]:
def introduce2(func):
    
    def inner(*args, **kwargs):
        print(func.__name__)
        return func(*args, **kwargs)
    
    inner.__name__ = func.__name__
    inner.__doc__ = func.__doc__
    
    return inner

In [18]:
@introduce2
def id2(x):
    '''Identity function'''
    return x
print(f'{help(id2)}\n----\n{id2.__name__}\n----\n{id2.__doc__}')

Help on function id2 in module __main__:

id2(*args, **kwargs)
    Identity function

None
----
id2
----
Identity function


The problem is that there is a lot of fields besides name and doc that you'd like to left untouched in some cases. But manually setting all these fields is kind of lame and repetitive. Solution: using decorator for a decorator.

In [19]:
from functools import wraps

def introduce3(func):
    
    @wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__)
        return func(*args, **kwargs)
    
    return inner

In [20]:
@introduce3
def id3(x):
    'Identity function'
    return x

In [21]:
print(f'{help(id3)}\n----\n{id3.__name__}\n----\n{id3.__doc__}')

Help on function id3 in module __main__:

id3(x)
    Identity function

None
----
id3
----
Identity function


And what if we want a decorator with parameters?

In [22]:
def introduce4(n=1):
    
    def res_decorator(func):
        
        @wraps(func)
        def inner(*args, **kwargs):
            print(('\n' + func.__name__) * n)
            return func(*args, **kwargs)
        
        return inner
    
    return res_decorator

In [23]:
@introduce4(n = 42)
def id4(x):
    'Identity function'
    return x

In [24]:
print(id4(20))


id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
id4
20


We just take our old decorator, add functionality that depends on some arguments that are not yet given. Then we add another layer: instead of having a decorator-function called `introduce`, we'll have a function called `introduce` that will seem to be a parameterized decorator, but instead it will take the arguments we wish to use and return the old `introduce` which could now use these arguments. For the sake of clarity, the returned function that is the actual decorator is called "resulting decorator".

If we pass no arguments, there's still a default value of n equal to 1. The only problem is that we can't just omit the brackets since introduce itself is not a decorator anymore, but a function that returns a decorator.

This works alright, but this number of nested layers makes it kind of unpleasant. Let's solve a decorator-designing problem with a decorator once again.

In [25]:
def parameterized(decorator):
    
    def decoFunction(*decargs, **deckwargs):
        
        def res_decorator(func):
            return decorator(func, *decargs, **deckwargs)
        
        return res_decorator
    
    return decoFunction

In [26]:
@parameterized
def introduce5(func, n=1):
    
    @wraps(func)
    def inner(*args, **kwargs):
        print(('\n' + func.__name__) * n)
        return func(*args, **kwargs)
    
    return inner

In [27]:
@introduce5(20)
def id5(x):
    'Identity function'
    return x

In [28]:
print(id5(42))


id5
id5
id5
id5
id5
id5
id5
id5
id5
id5
id5
id5
id5
id5
id5
id5
id5
id5
id5
id5
42


`parameterized` is a decorator that takes an initial decorator to make it a decorator with parameters.